In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fix_yahoo_finance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
import warnings
warnings.filterwarnings('ignore')
from pandas_datareader import data as pdr
import pandas
import yfinance as yfin
yfin.pdr_override()

# 1. Choosing Ticker and Training Window

In [4]:
Ticker = "MSFT"
Prediction_Date = "2020-05-27"
Training_Strat_Date = "2000-01-01"
Training_End_Date = "2020-05-26"

In [5]:
df1 = pdr.get_data_yahoo(Ticker, 
                         start=Training_Strat_Date,
                         end=Training_End_Date)

[*********************100%***********************]  1 of 1 completed


# 2. Presenting the Training Data

In [6]:
daily_prices1 = pd.concat([df1['Open'], df1['High'], df1['Low'],
                           df1['Adj Close'], df1['Volume'],df1['Close']], axis=1)

daily_prices1.columns = ['Open', 'High', 'Low',
                         'Adj Close','Volume','Close']
daily_prices1.head()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
1999-12-31,58.75000,58.8750,58.1250,36.777348,12517600,58.37500
2000-01-03,58.68750,59.3125,56.0000,36.718296,53228400,58.28125
2000-01-04,56.78125,58.5625,56.1250,35.477947,54119000,56.31250
2000-01-05,55.56250,58.1875,54.6875,35.852020,64059600,56.90625
2000-01-06,56.09375,56.9375,54.1875,34.651039,54976600,55.00000


In [7]:
daily_prices1.tail()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2020-05-18,185.750000,186.199997,183.960007,181.570862,35264500,184.910004
2020-05-19,185.029999,186.600006,183.490005,180.313980,26799100,183.630005
2020-05-20,184.809998,185.850006,183.940002,182.815063,31261300,185.660004
2020-05-21,185.399994,186.669998,183.289993,180.619217,29119500,183.429993
2020-05-22,183.190002,184.460007,182.539993,180.697998,20826900,183.509995


# 3. Creating 'Close t+1' Column

In [8]:
daily_prices1['Close t+1'] = 0

for i in range (len(daily_prices1['Close'])):
    daily_prices1['Close t+1'][i-1]=float(daily_prices1['Close'][i])

data1 = daily_prices1[daily_prices1.index != Training_End_Date]
data1.head()

,Open,High,Low,Adj Close,Volume,Close,Close t+1
Date,,,,,,,
1999-12-31,58.75000,58.8750,58.1250,36.777348,12517600,58.37500,58
2000-01-03,58.68750,59.3125,56.0000,36.718296,53228400,58.28125,56
2000-01-04,56.78125,58.5625,56.1250,35.477947,54119000,56.31250,56
2000-01-05,55.56250,58.1875,54.6875,35.852020,64059600,56.90625,55
2000-01-06,56.09375,56.9375,54.1875,34.651039,54976600,55.00000,55


In [9]:
data1.tail()

,Open,High,Low,Adj Close,Volume,Close,Close t+1
Date,,,,,,,
2020-05-18,185.750000,186.199997,183.960007,181.570862,35264500,184.910004,183
2020-05-19,185.029999,186.600006,183.490005,180.313980,26799100,183.630005,185
2020-05-20,184.809998,185.850006,183.940002,182.815063,31261300,185.660004,183
2020-05-21,185.399994,186.669998,183.289993,180.619217,29119500,183.429993,183
2020-05-22,183.190002,184.460007,182.539993,180.697998,20826900,183.509995,58


In [10]:
data1.describe()

,Open,High,Low,Adj Close,Volume,Close,Close t+1
count,5131.000000,5131.000000,5131.000000,5131.000000,5.131000e+03,5131.000000,5131.000000
mean,44.696361,45.159711,44.225597,37.817722,5.515151e+07,44.702231,44.211265
std,32.776598,33.085114,32.428960,34.122601,3.145628e+07,32.784144,32.787190
min,15.200000,15.620000,14.870000,11.487976,7.425600e+06,15.150000,15.000000
25%,26.600000,26.889999,26.309999,18.769629,3.304860e+07,26.590000,26.000000
50%,29.910000,30.200001,29.610001,22.047682,5.048160e+07,29.910000,29.000000
75%,47.040001,47.584999,46.555000,40.846437,6.872525e+07,47.070002,47.000000
max,190.649994,190.699997,186.470001,184.787720,5.910522e+08,188.699997,188.000000


# 4. Defining the Independent Variables and Dependent Variable

In [11]:
X = data1[['Open','High','Low','Adj Close','Volume']]
y = data1[['Close t+1']]

# 5. Building a Linear Regression Model

In [12]:
import statsmodels.api as sm
# Add a constant to the independent value
X1 = sm.add_constant(X)
# make regression model 
model = sm.OLS(y, X1)
# fit model and print results
result = model.fit()
print(result.summary())
lr=LinearRegression()
#fit the linear model on train data
lr_model=lr.fit(X,y)  

                            OLS Regression Results                            
Dep. Variable:              Close t+1   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 2.562e+05
Date:                Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                        23:31:15   Log-Likelihood:                -11012.
No. Observations:                5131   AIC:                         2.204e+04
Df Residuals:                    5125   BIC:                         2.208e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1478      0.124     -1.189      0.2

# 6. Evaluating the Model 

In [13]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
#predict using the model
y_pred=lr.predict(X)  
#calculate R squared
print("R squared of linear regression model:", "{:.2%}".format(lr.score(X, y)))
#calculate rmse
rmse=np.sqrt(mean_squared_error(y,y_pred))     
print('RMSE of linear regression model:',rmse)

R squared of linear regression model: 99.60%
RMSE of linear regression model: 2.069322720172566


# 7. Choosing Testing Window

In [14]:
df2 = pdr.get_data_yahoo([Ticker], start=Training_End_Date, end=Prediction_Date)

[*********************100%***********************]  1 of 1 completed


# 8. Presenting the Testing Data

In [15]:
daily_prices2 = pd.concat([df2['Open'], df2['High'], df2['Low'],
                           df2['Adj Close'], df2['Volume'],df2['Close']], axis=1)
daily_prices2.columns = ['Open', 'High', 'Low',
                         'Adj Close','Volume','Close']
daily_prices2

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2020-05-26,186.339996,186.5,181.100006,178.787735,36073600,181.570007


# 9. Creating 'Close t+1' Column Again

In [16]:
daily_prices2['Close t+1'] = 0.001

for i in range (len(daily_prices2['Close'])):
    daily_prices2['Close t+1'][i-1]=float(daily_prices2['Close'][i])
daily_prices2
data2 = daily_prices2[daily_prices2.index == Training_End_Date]
data2

,Open,High,Low,Adj Close,Volume,Close,Close t+1
Date,,,,,,,
2020-05-26,186.339996,186.5,181.100006,178.787735,36073600,181.570007,181.570007


# 10. Saving the True Close Price of the Prediction Date + 1

In [17]:
TRUE = data2["Close t+1"][0]
TRUE

181.57000732421875

# 11. Inserting the Prediction Date's Data into the Model

In [18]:
new_data = data2.drop(["Close t+1","Close"], axis=1)
new_data = np.array(new_data).reshape(1, -1)
new_pred=lr.predict(new_data)

# 12. Presenting the results

In [19]:
result = float(new_pred[0])
print("\033[1m The predicted close price of the stock on", Prediction_Date, "was: $",round(result,4))
print("\033[1m The true close price of the stock on", Prediction_Date, "was: $", round(TRUE,4))
aa = "{:.2%}".format(result/TRUE-1)
print("\033[1m The model missed by", aa)

 The predicted close price of the stock on 2020-05-27 was: $ 182.121
 The true close price of the stock on 2020-05-27 was: $ 181.57
 The model missed by 0.30%
